# Experiment 1 Baseline

Checking whether an array is a heap using an RNN

In [2]:
%cd ..
%pwd
import wandb
from experiments.heaps import *
from experiments.utils import *
from baselines.baseline_rnn import *
from torch_geometric.data import DataLoader
import matplotlib.pyplot as plt

/home/tekne/Oxford/ATML/Project


ModuleNotFoundError: No module named 'baselines'